In [7707]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

#Files to load
school_data_to_load = Path('Resources/schools_complete.csv')
student_data_to_load = Path('Resources/students_complete.csv')

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

Create a dataframe for the district displaying these key metrics
- Total number of unique schools
- Total students
- Total budget
- Average math score
- Average reading score
- % passing math (the percentage of students who passed math)
- % passing reading (the percentage of students who passed reading)
- % overall passing (the percentage of students who passed math AND reading)

In [7708]:
# Calculate the total number of unique schools
school_count = school_data_complete['school_name'].unique().size
school_count

15

In [7709]:
# Calculate the total number of students
student_count = len(school_data_complete.student_name)
student_count

39170

In [7710]:
# Calculate the total budget
total_budget = sum(school_data_complete.budget.unique())
total_budget

24649428

In [7711]:
# Calculate the average (mean) math score
average_math_score = school_data_complete.math_score.mean()
average_math_score

78.98537145774827

In [7712]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete.reading_score.mean()
average_reading_score

81.87784018381414

In [7713]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [7714]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [7715]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count / student_count * 100
overall_passing_rate

65.17232575950983

In [7716]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
data = {'Total Schools': [school_count], 'Total Students': [student_count], 'Total Budget': [total_budget], 'Average Math Score': [average_math_score], 'Average Reading Score': [average_reading_score], '% Passing Math': [passing_math_percentage], '% Passing Reading': [passing_reading_percentage], '% Overall Passing': [overall_passing_rate],}
district_summary = pd.DataFrame(data)

# Formatting
#district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
#district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [7717]:
# Use the code provided to select all of the school types
school_types = school_data_complete['type'].unique()
school_types

array(['District', 'Charter'], dtype=object)

In [7718]:
# Calculate the total student count per school
#rename "size" column to numstudents to avoid issues with size() 
school_data_complete.rename(columns = {'size':'numstudents'}, inplace = True)

per_school_counts = pd.DataFrame(school_data_complete[['school_name','numstudents']].value_counts())
per_school_counts = per_school_counts.reset_index()
per_school_counts

,school_name,numstudents,count
0,Bailey High School,4976,4976
1,Johnson High School,4761,4761
2,Hernandez High School,4635,4635
3,Rodriguez High School,3999,3999
4,Figueroa High School,2949,2949
5,Huang High School,2917,2917
6,Ford High School,2739,2739
7,Wilson High School,2283,2283
8,Cabrera High School,1858,1858
9,Wright High School,1800,1800


In [7719]:
# Calculate the total school budget
per_school_budget = school_data_complete[['school_name','budget']].copy()
per_school_budget = per_school_budget.drop_duplicates()
#per_school_budget = sum(per_school_budget.budget) #calculates total district budget (all schools)
per_school_budget

#calculate per capita spending per school
per_school_capita = school_data_complete[['school_name','budget', 'numstudents']]
per_school_capita = per_school_capita.drop_duplicates()
per_school_capita['per_capita_budget'] = (per_school_capita.budget/per_school_capita.numstudents)
per_school_capita

,school_name,budget,numstudents,per_capita_budget
0,Huang High School,1910635,2917,655.0
2917,Figueroa High School,1884411,2949,639.0
5866,Shelton High School,1056600,1761,600.0
7627,Hernandez High School,3022020,4635,652.0
12262,Griffin High School,917500,1468,625.0
13730,Wilson High School,1319574,2283,578.0
16013,Cabrera High School,1081356,1858,582.0
17871,Bailey High School,3124928,4976,628.0
22847,Holden High School,248087,427,581.0
23274,Pena High School,585858,962,609.0


In [7720]:
# Calculate the average test scores per school
per_school_math = pd.DataFrame(school_data_complete.groupby('school_name')['math_score'].mean()).copy()
per_school_math = per_school_math.reset_index()

per_school_reading = pd.DataFrame(school_data_complete.groupby('school_name')['reading_score'].mean()).copy()
per_school_reading = per_school_reading.reset_index()
per_school_reading

/Users/oliverkisza/Library/Python/3.11/lib/python/site-packages/IPython/core/displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


,school_name,reading_score
0,Bailey High School,81.033963
1,Cabrera High School,83.975780
2,Figueroa High School,81.158020
3,Ford High School,80.746258
4,Griffin High School,83.816757
5,Hernandez High School,80.934412
6,Holden High School,83.814988
7,Huang High School,81.182722
8,Johnson High School,80.966394
9,Pena High School,84.044699


In [7721]:
# Calculate the number of students per school with math scores of 70 or higher
#need to .value_counts() math_scores >= 70 and .groupby school_name 
school_students_passing_math = pd.DataFrame(school_data_complete.groupby('school_name')['math_score'].apply(lambda x: (x>=70).sum()).reset_index(name='pass_math_count'))
school_students_passing_math

,school_name,pass_math_count
0,Bailey High School,3318
1,Cabrera High School,1749
2,Figueroa High School,1946
3,Ford High School,1871
4,Griffin High School,1371
5,Hernandez High School,3094
6,Holden High School,395
7,Huang High School,1916
8,Johnson High School,3145
9,Pena High School,910


In [7722]:
# Calculate the number of students per school with reading scores of 70 or higher
school_students_passing_reading = school_data_complete.groupby('school_name')['reading_score'].apply(lambda x: (x>=70).sum()).reset_index(name='pass_reading_count')
school_students_passing_reading

,school_name,pass_reading_count
0,Bailey High School,4077
1,Cabrera High School,1803
2,Figueroa High School,2381
3,Ford High School,2172
4,Griffin High School,1426
5,Hernandez High School,3748
6,Holden High School,411
7,Huang High School,2372
8,Johnson High School,3867
9,Pena High School,923


In [7723]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = pd.DataFrame(school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
])

column_names = ['passing both']
school_students_passing_math_and_reading = pd.DataFrame(students_passing_math_and_reading.groupby(["school_name"]).size(), columns=column_names)
school_students_passing_math_and_reading = school_students_passing_math_and_reading.reset_index()
school_students_passing_math_and_reading

,school_name,passing both
0,Bailey High School,2719
1,Cabrera High School,1697
2,Figueroa High School,1569
3,Ford High School,1487
4,Griffin High School,1330
5,Hernandez High School,2481
6,Holden High School,381
7,Huang High School,1561
8,Johnson High School,2549
9,Pena High School,871


In [7724]:
# Use the provided code to calculate the passing rates
school_students_passing_math.set_index('school_name', inplace=True)
per_school_counts.set_index('school_name', inplace=True)
per_school_passing_math = pd.DataFrame(school_students_passing_math['pass_math_count']/per_school_counts['count']*100)
per_school_passing_math.rename(columns={0:'Percent_pass_math'}, inplace=True )
per_school_passing_math.reset_index(inplace=True)

school_students_passing_reading.set_index('school_name', inplace=True)
per_school_passing_reading = pd.DataFrame(school_students_passing_reading['pass_reading_count'] / per_school_counts['count'] * 100)
per_school_passing_reading.rename(columns={0:'Percent_pass_reading'}, inplace=True )
per_school_passing_reading.reset_index(inplace=True)

school_students_passing_math_and_reading.set_index('school_name', inplace=True)
overall_passing_rate = pd.DataFrame(school_students_passing_math_and_reading['passing both']/ per_school_counts['count'] * 100)
overall_passing_rate.rename(columns={0:'Percent_pass_both'}, inplace=True )
overall_passing_rate.reset_index(inplace=True)
overall_passing_rate

,school_name,Percent_pass_both
0,Bailey High School,54.642283
1,Cabrera High School,91.334769
2,Figueroa High School,53.204476
3,Ford High School,54.289887
4,Griffin High School,90.599455
5,Hernandez High School,53.527508
6,Holden High School,89.227166
7,Huang High School,53.513884
8,Johnson High School,53.539172
9,Pena High School,90.540541


In [7725]:

# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.merge(per_school_passing_math, per_school_passing_reading, on='school_name')
per_school_summary = pd.merge(per_school_summary, overall_passing_rate, on='school_name')

# Formatting
#per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
#per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,school_name,Percent_pass_math,Percent_pass_reading,Percent_pass_both
0,Bailey High School,66.680064,81.933280,54.642283
1,Cabrera High School,94.133477,97.039828,91.334769
2,Figueroa High School,65.988471,80.739234,53.204476
3,Ford High School,68.309602,79.299014,54.289887
4,Griffin High School,93.392371,97.138965,90.599455
5,Hernandez High School,66.752967,80.862999,53.527508
6,Holden High School,92.505855,96.252927,89.227166
7,Huang High School,65.683922,81.316421,53.513884
8,Johnson High School,66.057551,81.222432,53.539172
9,Pena High School,94.594595,95.945946,90.540541


## Highest-Performing Schools (by % Overall Passing)

In [7726]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(by='Percent_pass_both', ascending=False)
top_schools.head(5)

,school_name,Percent_pass_math,Percent_pass_reading,Percent_pass_both
1,Cabrera High School,94.133477,97.039828,91.334769
12,Thomas High School,93.272171,97.308869,90.948012
4,Griffin High School,93.392371,97.138965,90.599455
13,Wilson High School,93.867718,96.539641,90.582567
9,Pena High School,94.594595,95.945946,90.540541


## Bottom Performing Schools (by % Overall Passing)

In [7727]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(by='Percent_pass_both', ascending=True)
bottom_schools.head(5)

,school_name,Percent_pass_math,Percent_pass_reading,Percent_pass_both
10,Rodriguez High School,66.366592,80.220055,52.988247
2,Figueroa High School,65.988471,80.739234,53.204476
7,Huang High School,65.683922,81.316421,53.513884
5,Hernandez High School,66.752967,80.862999,53.527508
8,Johnson High School,66.057551,81.222432,53.539172


## Math Scores by Grade

In [7728]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = pd.DataFrame(ninth_graders.groupby('school_name')['math_score'].mean())
tenth_grader_math_scores = pd.DataFrame(tenth_graders.groupby('school_name')['math_score'].mean())
eleventh_grader_math_scores = pd.DataFrame(eleventh_graders.groupby('school_name')['math_score'].mean())
twelfth_grader_math_scores = pd.DataFrame(twelfth_graders.groupby('school_name')['math_score'].mean())

ninth_grade_math_scores.reset_index(inplace=True)
tenth_grader_math_scores.reset_index(inplace=True)
eleventh_grader_math_scores.reset_index(inplace=True)
twelfth_grader_math_scores.reset_index(inplace=True)
# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
tomerge = pd.merge(ninth_grade_math_scores, tenth_grader_math_scores, on='school_name')
toalsomerge = pd.merge(eleventh_grader_math_scores, twelfth_grader_math_scores, on='school_name')
math_scores_by_grade = pd.merge(tomerge, toalsomerge, on='school_name')

# Minor data wrangling
math_scores_by_grade.rename(columns = {'math_score_x_x':'9th', 'math_score_y_x':'10th', 'math_score_x_y':'11th', 'math_score_y_y':'12th'}, inplace = True)
# Display the DataFrame
math_scores_by_grade

,school_name,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade

In [7729]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = pd.DataFrame(ninth_graders.groupby('school_name')['reading_score'].mean())
tenth_grader_reading_scores = pd.DataFrame(tenth_graders.groupby('school_name')['reading_score'].mean())
eleventh_grader_reading_scores = pd.DataFrame(eleventh_graders.groupby('school_name')['reading_score'].mean())
twelfth_grader_reading_scores = pd.DataFrame(twelfth_graders.groupby('school_name')['reading_score'].mean())

ninth_grade_reading_scores.reset_index(inplace=True)
tenth_grader_reading_scores.reset_index(inplace=True)
eleventh_grader_reading_scores.reset_index(inplace=True)
twelfth_grader_reading_scores.reset_index(inplace=True)

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
readtomerge = pd.merge(ninth_grade_reading_scores, tenth_grader_reading_scores, on='school_name')
readtoalsomerge = pd.merge(eleventh_grader_reading_scores, twelfth_grader_reading_scores, on='school_name')
reading_scores_by_grade = pd.merge(readtomerge, readtoalsomerge, on='school_name')

# Minor data wrangling
reading_scores_by_grade.rename(columns = {'reading_score_x_x':'9th', 'reading_score_y_x':'10th', 'reading_score_x_y':'11th', 'reading_score_y_y':'12th'}, inplace = True)

# Display the DataFrame
reading_scores_by_grade

,school_name,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [7730]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [7731]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()


In [7732]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita['per_capita_budget'], bins=spending_bins)
school_spending_df

,school_name,Percent_pass_math,Percent_pass_reading,Percent_pass_both,Spending Ranges (Per Student)
0,Bailey High School,66.680064,81.933280,54.642283,"(645.0, 680.0]"
1,Cabrera High School,94.133477,97.039828,91.334769,NaN
2,Figueroa High School,65.988471,80.739234,53.204476,NaN
3,Ford High School,68.309602,79.299014,54.289887,NaN
4,Griffin High School,93.392371,97.138965,90.599455,NaN
5,Hernandez High School,66.752967,80.862999,53.527508,NaN
6,Holden High School,92.505855,96.252927,89.227166,NaN
7,Huang High School,65.683922,81.316421,53.513884,NaN
8,Johnson High School,66.057551,81.222432,53.539172,NaN
9,Pena High School,94.594595,95.945946,90.540541,NaN


In [7733]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

KeyError: 'Column not found: Average Math Score'

In [ ]:
# Assemble into DataFrame
spending_summary = 

# Display results
spending_summary

## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = 
per_school_summary

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = 

# Display results
size_summary

## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = 

# Display results
type_summary